In [38]:
import numpy as np
from astropy.coordinates import SkyCoord, Angle, EarthLocation, Longitude
from astropy.time import Time
from astropy import units

In [2]:
ra_hrs = 12.1
dec_degs = -42.3
mjd = 55780.1
latitude = Angle('-26d42m11.94986s')
longitude = Angle('116d40m14.93485s')

In [3]:
print(latitude.rad)
print(longitude.rad)

-0.466060844839
2.03628986686


In [4]:
obs_time = Time(mjd, format='mjd', location = (longitude, latitude))
lst_mean = obs_time.sidereal_time('mean')
lst_apparent = obs_time.sidereal_time('apparent')
print(lst_mean.rad)
print(lst_apparent.rad)

1.88387679923
1.88395723805


In [5]:
mwa_tools_lst_mean = 1.8838984

In [6]:
print(Angle(lst_mean.rad - mwa_tools_lst_mean, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(lst_apparent.rad - mwa_tools_lst_mean, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg00m04.4555s
0deg00m12.1362s


# -> mean LST disagrees by ~4.5 arcseconds (should we use apparent??)

In [7]:
icrs_coord = SkyCoord(ra=Angle(ra_hrs, unit='hr'), dec=Angle(dec_degs, unit='deg'),
                      obstime=Time(mjd, format='mjd'), equinox=Time(mjd, format='mjd'))
icrs_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (181.5, -42.3)>

In [8]:
icrs_coord.ra.rad, icrs_coord.dec.rad

(3.1677725923697078, -0.7382742735936013)

In [10]:
gcrs_coord = icrs_coord.transform_to('gcrs')
gcrs_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec) in deg
    (181.49525535, -42.3015925)>

In [11]:
cirs_coord.ra.rad, cirs_coord.dec.rad

(3.16772535331374, -0.7394629729188962)

In [12]:
gcrs_coord.ra.rad, gcrs_coord.dec.rad

(3.167689782620545, -0.7383020680162412)

In [14]:
# This is the change in ra & dec between icrs & cirs
print(icrs_coord.ra-cirs_coord.ra)
print(icrs_coord.dec-cirs_coord.dec)

0d00m09.7438s
0d04m05.1868s


In [15]:
# This is the change in ra & dec between icrs & gcrs
print(icrs_coord.ra-gcrs_coord.ra)
print(icrs_coord.dec-gcrs_coord.dec)

0d00m17.0807s
0d00m05.733s


In [16]:
hour_angle_mean = lst_mean.rad - cirs_coord.ra.rad
hour_angle_apparent = lst_apparent.rad - cirs_coord.ra.rad

print(hour_angle_mean)
print(hour_angle_apparent)

-1.28384855408
-1.28376811527


In [17]:
# mwa_tools comp2
# It turns out these are not actually used in the return values at all. So don't worry about them
mwa_tools_comp2_ra = 3.1703987
mwa_tools_comp2_dec = -0.73946297

In [18]:
# compare mwa_tools comp2 to cirs
print(Angle(cirs_coord.ra.rad - mwa_tools_comp2_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(cirs_coord.dec.rad - mwa_tools_comp2_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg09m11.4173s
-0deg00m00.0006s


In [19]:
# compare mwa_tools comp2 to gcrs
print(Angle(gcrs_coord.ra.rad - mwa_tools_comp2_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(gcrs_coord.dec.rad - mwa_tools_comp2_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg09m18.7543s
0deg03m59.4532s


In [20]:
# mwa_tools comp3
mwa_tools_comp3_ra = 3.1676898
mwa_tools_comp3_dec = -0.73830205

In [21]:
# compare mwa_tools comp3 to cirs
print(Angle(cirs_coord.ra.rad - mwa_tools_comp3_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(cirs_coord.dec.rad - mwa_tools_comp3_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

0deg00m07.3334s
-0deg03m59.4575s


In [22]:
# compare mwa_tools comp3 to gcrs
print(Angle(gcrs_coord.ra.rad - mwa_tools_comp3_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(gcrs_coord.dec.rad - mwa_tools_comp3_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg00m00.0036s
-0deg00m00.0037s


# -> mwa_tools ra_aber/dec_aber (calc3) agrees with GCRS, meaning that it accounts for aberration but not nutation or precession

In [53]:
# The frame radio astronomers call the apparent or current epoch is the
# "true equator & equinox" frame, notated E_gamma in the USNO circular
# astropy doesn't have this frame but it's pretty easy to adapt the CIRS frame
# by modifying the ra to reflect the difference between
# GAST (Grenwich Apparent Sidereal Time) and the earth rotation angle (theta)
def egamma_to_cirs_ra(egamma_ra, time):
    from astropy import _erfa as erfa
    from astropy.coordinates.builtin_frames.utils import get_jd12
    era = erfa.era00(*get_jd12(Time(mjd, format='mjd'), 'ut1'))
    theta_earth = Angle(era, unit='rad')

    assert(isinstance(time, Time))
    gast = time.sidereal_time('apparent', longitude=0)
    cirs_ra = e_gamma_ra - (gast-theta_earth)
    return cirs_ra

In [56]:
# now get equivilents of lst, lat in GCRS (rather than precessed) frame:
# effectively current zenith "unprecessed" to J2000 but not undoing aberration
# mwa_tools calls these "lmst2000" and "newarrlat"
# also need what they call "ha2000" which is "lmst2000" - apparent ra

loc_obj = EarthLocation.from_geodetic(lon=longitude, lat=latitude)

# use CIRS but with a different ra to account for the difference between LST & earth's rotation angle
cirs_ra = egamma_to_cirs_ra(lst_apparent, Time(mjd, format='mjd'))

egamma_zenith_coord = SkyCoord(ra=cirs_ra, dec=latitude, frame='cirs',
                               obstime=Time(mjd, format='mjd'), location = loc_obj)
egamma_zenith_coord

<SkyCoord (CIRS: obstime=55780.1): (ra, dec) in deg
    (107.78961213, -26.70331941)>

In [57]:
# check where it is in altaz (should be near zenith):
egamma_zenith_altaz = zenith_coord.transform_to('altaz')
egamma_zenith_altaz

<SkyCoord (AltAz: obstime=55780.1, location=(-2559302.5737783727, 5095070.526830904, -2848887.400942108) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    (28.60732437, 89.99985797)>

In [58]:
(egamma_zenith_altaz.alt - Angle('90d')).to_string(unit=units.degree, sep=('deg', 'm', 's'))

u'-0deg00m00.5113s'

In [59]:
gcrs_zenith = egamma_zenith_altaz.transform_to('gcrs')
gcrs_zenith

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec) in deg
    (107.82139279, -26.68249473)>

In [60]:
print(gcrs_zenith.ra.rad)
print(gcrs_zenith.dec.rad)

1.88183830833
-0.465697385751


In [63]:
# This is the change between egamma & gcrs
print(gcrs_zenith.ra - lst_apparent)
print(gcrs_zenith.dec - egamma_zenith_coord.dec)

-0d07m17.0606s
0d01m14.9688s


In [77]:
hour_angle2000 = Longitude(gcrs_zenith.ra - gcrs_coord.ra)
hour_angle2000

<Longitude 286.32613744 deg>

In [78]:
# for these, I used the apparent lst (and aberration corrected ra/decs) from this notebook,
# so that's not a source of error
mwa_tools_comp4_newarrlat = -0.46569739
mwa_tools_comp4_lmst2000 = 1.8818386
mwa_tools_comp4_ha2000 = 4.9973341

In [79]:
# This is what the MWA tools code gives as the change between precessed & not
print(Angle(lst_apparent.rad - mwa_tools_comp4_lmst2000, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(latitude.rad - mwa_tools_comp4_newarrlat, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

0deg07m17.0005s
-0deg01m14.9679s


In [80]:
print(Angle(gcrs_zenith.ra.rad - mwa_tools_comp4_lmst2000, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(gcrs_zenith.dec.rad - mwa_tools_comp4_newarrlat, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(hour_angle2000.rad - mwa_tools_comp4_ha2000, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg00m00.0602s
0deg00m00.0009s
-0deg00m00.0551s


# "unprecessed" (= CGRS) values disagree by ~13 arcseconds

In [37]:
# now define an antenna position to work on uvw rotation
# in east/north/up frame (relative to array center) in meters:
x_ant = -101.94
y_ant = 0156.41
z_ant = 0001.24